# Sparse GP Regression

### 14th January 2014 James Hensman
#### 29th September 2014 Neil Lawrence (added sub-titles, notes and some references).

This example shows the variational compression effect of so-called 'sparse' Gaussian processes. In particular we show how using the variational free energy framework of [Titsias, 2009](http://jmlr.csail.mit.edu/proceedings/papers/v5/titsias09a/titsias09a.pdf) we can compress a Gaussian process fit. First we set up the notebook with a fixed random seed, and import GPy.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import GPy
import numpy as np
np.random.seed(101)

## Sample Function

Now we'll sample a Gaussian process regression problem directly from a Gaussian process prior. We'll use an exponentiated quadratic covariance function with a lengthscale and variance of 1 and sample 50 equally spaced points. 

In [ ]:
N = 50
noise_var = 0.05

X = np.linspace(0,10,50)[:,None]
k = GPy.kern.RBF(1)
y = np.random.multivariate_normal(np.zeros(N),k.K(X)+np.eye(N)*np.sqrt(noise_var)).reshape(-1,1)

## Full Gaussian Process Fit

Now we use GPy to optimize the parameters of a Gaussian process given the sampled data. Here, there are no approximations, we simply fit the full Gaussian process.

In [ ]:
m_full = GPy.models.GPRegression(X,y)
m_full.optimize('bfgs')
m_full.plot()
print(m_full)

## A Poor `Sparse' GP Fit

Now we construct a sparse Gaussian process. This model uses the inducing variable approximation and initialises the inducing variables in two 'clumps'. Our initial fit uses the *correct* covariance function parameters, but a badly placed set of inducing points. 

In [ ]:
Z = np.hstack((np.linspace(2.5,4.,3),np.linspace(7,8.5,3)))[:,None]
m = GPy.models.SparseGPRegression(X,y,Z=Z)
m.likelihood.variance = noise_var
m.plot()
print(m)


Notice how the fit is reasonable where there are inducing points, but bad elsewhere. 

### Optimizing Covariance Parameters

Next, we will try and find the optimal covariance function parameters, given that the inducing inputs are held in their current location. 

In [ ]:
m.inducing_inputs.fix()
m.optimize('bfgs')
m.plot()
print(m)

The poor location of the inducing inputs causes the model to 'underfit' the data. The lengthscale is much longer than the full GP, and the noise variance is larger. This is because in this case the Kullback Leibler term in the objective free energy is dominating, and requires a larger lengthscale to improve the quality of the approximation. This is due to the poor location of the inducing inputs. 

### Optimizing Inducing Inputs

Firstly we try optimzing the location of the inducing inputs to fix the problem, however we still get a larger lengthscale than the Gaussian process we sampled from (or the full GP fit we did at the beginning).

In [ ]:
m.randomize()
m.Z.unconstrain()
m.optimize('bfgs')
m.plot()

The inducing points spread out to cover the data space, but the fit isn't quite there. We can try increasing the number of the inducing points.

### Train with More Inducing Points

Now we try 12 inducing points, rather than the original six. We then compare with the full Gaussian process likelihood.

In [ ]:
Z = np.random.rand(12,1)*12
m = GPy.models.SparseGPRegression(X,y,Z=Z)

m.optimize('bfgs')
m.plot()
m_full.plot()
print(m.log_likelihood(), m_full.log_likelihood())

This time, we have enough inducing points and the fit resembles that of the GP. This is verified by the fact that the bound on the marginal likelihood is tight, which means that our variational approximation must be good (the difference between the bound and the true likelihood is the Kullback Leibler divergence between the approximation and the truth). 

## 2D GP regression

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# True distribution function
c1 = np.array([0.2, 0.3])
c2 = np.array([0.8, 0.5])

def f(x, y):
    return np.maximum(np.exp(-np.sum((np.hstack((x, y)) - c1)**2, 1)), np.exp(-np.sum((np.hstack((x, y)) - c2)**2, 1)))

# Plot the function
x = np.linspace(0, 1, 100)
y = np.linspace(0, 1, 100)
X, Y = np.meshgrid(x, y)
Z = f(X.reshape(-1,1), Y.reshape(-1,1)).reshape(X.shape)
plt.contour(X, Y, Z, 20)

In [ ]:
# Fit a 2D function f(x,y) = c over [0,1]x[0,1] on data (x,y,c)

N = 100  # number of data points

# Sample random points in the input space
X = np.random.rand(N, 2)
k = GPy.kern.RBF(2)
# Sample some noiseless data
C = f(X[:,0:1], X[:,1:2])

# Create a GPy model
m = GPy.models.GPRegression(X, C, kernel=k)
m.optimize('bfgs')
m.plot()
print(m)

In [ ]:
import numpy as np
np.set_printoptions(precision=2, suppress=True)

In [ ]:
R = np.array([[0, 0, -1], 
              [-1, 0, 0], 
              [0, 1, 0]])

R.T @ R

No mapping

In [ ]:
black_cam = np.array([-2.220446049250313e-16,1,1.1102230246251565e-16,0,-1.1102230246251565e-16,-2.220446049250313e-16,1,0,1,1.1102230246251565e-16,0,0,0.9999999999999986,-0.3333333432674409,1,1])
print(black_cam.reshape(4,4).T)
""
white_cam = np.array([2.220446049250313e-16,0.9999999999999999,-1.1102230246251565e-16,0,-3.3306690738754696e-16,2.220446049250313e-16,0.9999999999999999,0,0.9999999999999999,-1.1102230246251565e-16,4.440892098500626e-16,0,0.9999999999999991,0.3333333432674407,1.0000000000000002,1])
print(white_cam.reshape(4,4).T)

Mapping

In [ ]:
black_cam = np.array([1,0,0,0,0,2.220446049250313e-16,1,0,0,-1,2.220446049250313e-16,0,-0.3333333432674408,-0.6666666865348804,-1.0000000000000002,1])
print(black_cam.reshape(4,4).T)

white_cam = np.array([1,0,0,0,0,2.220446049250313e-16,1,0,0,-1,2.220446049250313e-16,0,1,0.6666666865348823,0.3333333432674406,1])
print(white_cam.reshape(4,4).T)